In [2]:
import os
import json
import pandas as pd

class MatchDataExtractor:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.all_deliveries = []

    def extract_all_matches(self):
        """Extract deliveries from all JSON files in the folder."""
        for filename in os.listdir(self.folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(self.folder_path, filename)
                self.extract_from_file(file_path, filename)

    def extract_from_file(self, file_path, filename):
        """Extract deliveries from a single match JSON file."""
        with open(file_path, 'r', encoding="utf-8") as f:
            data = json.load(f)

        for inning in data.get("innings", []):
            # Handle Cricsheet format type 1 (wrapped in "1st innings", "2nd innings")
            if len(inning.keys()) == 1 and isinstance(list(inning.values())[0], dict):
                inning_data = list(inning.values())[0]
            # Handle format type 2 (team/overs directly in inning dict)
            else:
                inning_data = inning

            team = inning_data.get("team")
            for over in inning_data.get("overs", []):
                over_number = over.get("over")
                for delivery in over.get("deliveries", []):
                    self.all_deliveries.append(
                        self.flatten_delivery(delivery, team, over_number, filename)
                    )

    def flatten_delivery(self, delivery, team, over_number, filename):
        """Flatten delivery dictionary into a row."""
        extras_type = list(delivery.get("extras", {}).keys())
        extras_type_str = extras_type[0] if extras_type else None

        delivery_flat = {
            "match_file": filename,
            "team": team,
            "over": over_number,
            "batter": delivery.get("batter"),
            "bowler": delivery.get("bowler"),
            "non_striker": delivery.get("non_striker"),
            "runs_batter": delivery["runs"]["batter"],
            "runs_extras": delivery["runs"]["extras"],
            "runs_total": delivery["runs"]["total"],
            "extras_type": extras_type_str,
            "wicket_kind": None,
            "wicket_player_out": None,
            "wicket_fielders": []
        }

        if "wickets" in delivery:
            for wicket in delivery["wickets"]:
                delivery_flat["wicket_kind"] = wicket.get("kind")
                delivery_flat["wicket_player_out"] = wicket.get("player_out")
                fielders = wicket.get("fielders", [])
                delivery_flat["wicket_fielders"] = [
                    f.get("name") for f in fielders if isinstance(f, dict) and "name" in f
                ]

        return delivery_flat

    def to_dataframe(self):
        """Convert all deliveries to a Pandas DataFrame."""
        return pd.DataFrame(self.all_deliveries)


if __name__ == "__main__":
    folder = "/Users/rangasamyelancheziyan/Desktop/cricket/tests_json"  

    extractor = MatchDataExtractor(folder)
    extractor.extract_all_matches()
    df_test = extractor.to_dataframe()  # DataFrame directly

    print(df_test.head())  # Show first rows


     match_file   team  over      batter    bowler non_striker  runs_batter  \
0  1223869.json  India     0     PP Shaw  MA Starc  MA Agarwal            0   
1  1223869.json  India     0     PP Shaw  MA Starc  MA Agarwal            0   
2  1223869.json  India     0   CA Pujara  MA Starc  MA Agarwal            0   
3  1223869.json  India     0   CA Pujara  MA Starc  MA Agarwal            1   
4  1223869.json  India     0  MA Agarwal  MA Starc   CA Pujara            0   

   runs_extras  runs_total extras_type wicket_kind wicket_player_out  \
0            0           0        None        None              None   
1            0           0        None      bowled           PP Shaw   
2            0           0        None        None              None   
3            0           1        None        None              None   
4            0           0        None        None              None   

  wicket_fielders  
0              []  
1              []  
2              []  
3           

In [7]:
import os
import json
import pandas as pd
import mysql.connector


class MatchDataExtractor:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.all_deliveries = []

    def extract_all_matches(self):
        """Extract deliveries from all JSON files in the folder."""
        for filename in os.listdir(self.folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(self.folder_path, filename)
                self.extract_from_file(file_path, filename)

    def extract_from_file(self, file_path, filename):
        """Extract deliveries from a single match JSON file."""
        with open(file_path, 'r', encoding="utf-8") as f:
            data = json.load(f)

        for inning in data.get("innings", []):
            # Detect format type
            if "team" in inning:
                inning_data = inning
            else:
                inning_data = list(inning.values())[0]

            team = inning_data.get("team")
            for over in inning_data.get("overs", []):
                over_number = over.get("over")
                for delivery in over.get("deliveries", []):
                    self.all_deliveries.append(
                        self.flatten_delivery(delivery, team, over_number, filename)
                    )

    def flatten_delivery(self, delivery, team, over_number, filename):
        """Flatten delivery dictionary into a row."""
        extras_type = list(delivery.get("extras", {}).keys())
        extras_type_str = extras_type[0] if extras_type else None

        delivery_flat = {
            "match_file": filename,
            "team": team,
            "over_no": over_number,  
            "batter": delivery.get("batter"),
            "bowler": delivery.get("bowler"),
            "non_striker": delivery.get("non_striker"),
            "runs_batter": delivery["runs"]["batter"],
            "runs_extras": delivery["runs"]["extras"],
            "runs_total": delivery["runs"]["total"],
            "extras_type": extras_type_str,
            "wicket_kind": None,
            "wicket_player_out": None,
            "wicket_fielders": []
        }

        if "wickets" in delivery:
            for wicket in delivery["wickets"]:
                delivery_flat["wicket_kind"] = wicket.get("kind")
                delivery_flat["wicket_player_out"] = wicket.get("player_out")
                fielders = wicket.get("fielders", [])
                delivery_flat["wicket_fielders"] = [
                    f.get("name") for f in fielders if isinstance(f, dict) and "name" in f
                ]

        return delivery_flat

    def to_dataframe(self):
        """Convert all deliveries to a Pandas DataFrame."""
        return pd.DataFrame(self.all_deliveries)




class MySQLInserter:
    def __init__(self, host, user, password, database):
        self.conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database
        )
        self.cursor = self.conn.cursor()

    def create_table(self):
        """Create deliveries table in MySQL."""
        create_table_query = """
        CREATE TABLE IF NOT EXISTS deliveries (
            id INT AUTO_INCREMENT PRIMARY KEY,
            match_file VARCHAR(255),
            team VARCHAR(100),
            over_no INT,
            batter VARCHAR(100),
            bowler VARCHAR(100),
            non_striker VARCHAR(100),
            runs_batter INT,
            runs_extras INT,
            runs_total INT,
            extras_type VARCHAR(50),
            wicket_kind VARCHAR(50),
            wicket_player_out VARCHAR(100),
            wicket_fielders TEXT
        );
        """
        self.cursor.execute(create_table_query)
        self.conn.commit()

    def insert_dataframe(self, df):
        """Insert DataFrame into MySQL table."""
        insert_query = """
        INSERT INTO deliveries (
            match_file, team, over_no, batter, bowler, non_striker,
            runs_batter, runs_extras, runs_total, extras_type,
            wicket_kind, wicket_player_out, wicket_fielders
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        for _, row in df.iterrows():
            self.cursor.execute(insert_query, (
                row["match_file"],
                row["team"],
                row["over_no"],
                row["batter"],
                row["bowler"],
                row["non_striker"],
                row["runs_batter"],
                row["runs_extras"],
                row["runs_total"],
                row["extras_type"],
                row["wicket_kind"],
                row["wicket_player_out"],
                ",".join(row["wicket_fielders"]) if row["wicket_fielders"] else None
            ))
        self.conn.commit()

    def close(self):
        """Close MySQL connection."""
        self.cursor.close()
        self.conn.close()



if __name__ == "__main__":
   
    extractor = MatchDataExtractor(folder_path="/Users/rangasamyelancheziyan/Desktop/cricket/tests_json")
    extractor.extract_all_matches()
    df = extractor.to_dataframe()

    print("Sample DataFrame:")
    print(df.head())

   
    inserter = MySQLInserter(
        host="localhost",
        user="root",
        password="Sheela@1981",
        database="crickets_db"
    )
    inserter.create_table()
    inserter.insert_dataframe(df)
    inserter.close()

    print("✅ Data inserted into MySQL successfully!")


Sample DataFrame:
     match_file   team  over_no      batter    bowler non_striker  \
0  1223869.json  India        0     PP Shaw  MA Starc  MA Agarwal   
1  1223869.json  India        0     PP Shaw  MA Starc  MA Agarwal   
2  1223869.json  India        0   CA Pujara  MA Starc  MA Agarwal   
3  1223869.json  India        0   CA Pujara  MA Starc  MA Agarwal   
4  1223869.json  India        0  MA Agarwal  MA Starc   CA Pujara   

   runs_batter  runs_extras  runs_total extras_type wicket_kind  \
0            0            0           0        None        None   
1            0            0           0        None      bowled   
2            0            0           0        None        None   
3            1            0           1        None        None   
4            0            0           0        None        None   

  wicket_player_out wicket_fielders  
0              None              []  
1           PP Shaw              []  
2              None              []  
3           

In [10]:
import os
import json
import pandas as pd
import mysql.connector

class MatchDataExtractor:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.odi_matches = []

    def extract_all_matches(self):
        """Extract deliveries from all JSON files in the folder."""
        for filename in os.listdir(self.folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(self.folder_path, filename)
                self.extract_from_file(file_path, filename)

    def extract_from_file(self, file_path, filename):
        """Extract deliveries from a single match JSON file."""
        with open(file_path, 'r', encoding="utf-8") as f:
            data = json.load(f)

        for inning in data.get("innings", []):
            inning_data = inning if "team" in inning else list(inning.values())[0]
            team = inning_data.get("team")
            for over in inning_data.get("overs", []):
                over_number = over.get("over")
                for delivery in over.get("deliveries", []):  # ✅ FIXED key
                    self.odi_matches.append(
                        self.flatten_delivery(delivery, team, over_number, filename)
                    )

    def flatten_delivery(self, delivery, team, over_number, filename):
        """Flatten delivery dictionary into a row."""
        extras_type = list(delivery.get("extras", {}).keys())
        extras_type_str = extras_type[0] if extras_type else None

        delivery_flat = {
            "match_file": filename,
            "team": team,
            "over_no": over_number,  # ✅ safe for MySQL
            "batter": delivery.get("batter"),
            "bowler": delivery.get("bowler"),
            "non_striker": delivery.get("non_striker"),
            "runs_batter": delivery["runs"]["batter"],
            "runs_extras": delivery["runs"]["extras"],
            "runs_total": delivery["runs"]["total"],
            "extras_type": extras_type_str,
            "wicket_kind": None,
            "wicket_player_out": None,
            "wicket_fielders": []
        }

        if "wickets" in delivery:
            for wicket in delivery["wickets"]:
                delivery_flat["wicket_kind"] = wicket.get("kind")
                delivery_flat["wicket_player_out"] = wicket.get("player_out")
                fielders = wicket.get("fielders", [])
                if isinstance(fielders, list):
                    delivery_flat["wicket_fielders"] = [
                        f.get("name") for f in fielders if isinstance(f, dict) and "name" in f
                    ]

        return delivery_flat

    def to_dataframe(self):
        """Convert all deliveries to a Pandas DataFrame."""
        return pd.DataFrame(self.odi_matches)


class MySQLInserter:
    def __init__(self, host, user, password, database):
        self.conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            charset='utf8mb4'
        )
        self.cursor = self.conn.cursor()

    def create_table(self):
        """Create odi_matches table in MySQL."""
        create_table_query = """
        CREATE TABLE IF NOT EXISTS odi_matches (
            id INT AUTO_INCREMENT PRIMARY KEY,
            match_file VARCHAR(255),
            team VARCHAR(100),
            over_no INT,
            batter VARCHAR(100),
            bowler VARCHAR(100),
            non_striker VARCHAR(100),
            runs_batter INT,
            runs_extras INT,
            runs_total INT,
            extras_type VARCHAR(50),
            wicket_kind VARCHAR(50),
            wicket_player_out VARCHAR(100),
            wicket_fielders TEXT
        );
        """
        self.cursor.execute(create_table_query)
        self.conn.commit()

    def insert_dataframe(self, df):
        """Insert DataFrame into MySQL table."""
        insert_query = """
        INSERT INTO odi_matches (
            match_file, team, over_no, batter, bowler, non_striker,
            runs_batter, runs_extras, runs_total, extras_type,
            wicket_kind, wicket_player_out, wicket_fielders
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        for _, row in df.iterrows():
            self.cursor.execute(insert_query, (
                row["match_file"],
                row["team"],
                row["over_no"],
                row["batter"],
                row["bowler"],
                row["non_striker"],
                row["runs_batter"],
                row["runs_extras"],
                row["runs_total"],
                row["extras_type"],
                row["wicket_kind"],
                row["wicket_player_out"],
                ",".join(row["wicket_fielders"]) if isinstance(row["wicket_fielders"], list) else None
            ))
        self.conn.commit()

    def close(self):
        """Close MySQL connection."""
        self.cursor.close()
        self.conn.close()


if __name__ == "__main__":
    extractor = MatchDataExtractor(folder_path="/Users/rangasamyelancheziyan/Desktop/cricket/odis_json")
    extractor.extract_all_matches()
    df = extractor.to_dataframe()
    print(df.head())  

    inserter = MySQLInserter(
        host="localhost",
        user="root",
        password="Sheela@1981",
        database="crickets_db"
    )
    inserter.create_table()
    inserter.insert_dataframe(df)
    inserter.close()

    print("✅ Data inserted into MySQL successfully!")


     match_file   team  over_no       batter    bowler      non_striker  \
0  1387601.json  India        0   RD Gaikwad  N Burger  B Sai Sudharsan   
1  1387601.json  India        0   RD Gaikwad  N Burger  B Sai Sudharsan   
2  1387601.json  India        0  Tilak Varma  N Burger  B Sai Sudharsan   
3  1387601.json  India        0  Tilak Varma  N Burger  B Sai Sudharsan   
4  1387601.json  India        0  Tilak Varma  N Burger  B Sai Sudharsan   

   runs_batter  runs_extras  runs_total extras_type wicket_kind  \
0            4            0           4        None        None   
1            0            0           0        None         lbw   
2            0            0           0        None        None   
3            0            0           0        None        None   
4            0            0           0        None        None   

  wicket_player_out wicket_fielders  
0              None              []  
1        RD Gaikwad              []  
2              None             

In [12]:
import os
import json
import pandas as pd
import mysql.connector

class MatchDataExtractor:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.t20_matches = []

    def extract_all_matches(self):
        """Extract deliveries from all JSON files in the folder."""
        for filename in os.listdir(self.folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(self.folder_path, filename)
                self.extract_from_file(file_path, filename)

    def extract_from_file(self, file_path, filename):
        """Extract deliveries from a single match JSON file."""
        with open(file_path, 'r', encoding="utf-8") as f:
            data = json.load(f)

        for inning in data.get("innings", []):
            inning_data = inning if "team" in inning else list(inning.values())[0]
            team = inning_data.get("team")
            for over in inning_data.get("overs", []):
                over_number = over.get("over")
                for delivery in over.get("deliveries", []):  
                    self.t20_matches.append(
                        self.flatten_delivery(delivery, team, over_number, filename)
                    )

    def flatten_delivery(self, delivery, team, over_number, filename):
        """Flatten delivery dictionary into a row."""
        extras_type = list(delivery.get("extras", {}).keys())
        extras_type_str = extras_type[0] if extras_type else None

        delivery_flat = {
            "match_file": filename,
            "team": team,
            "over_no": over_number,  # ✅ safe for MySQL
            "batter": delivery.get("batter"),
            "bowler": delivery.get("bowler"),
            "non_striker": delivery.get("non_striker"),
            "runs_batter": delivery["runs"]["batter"],
            "runs_extras": delivery["runs"]["extras"],
            "runs_total": delivery["runs"]["total"],
            "extras_type": extras_type_str,
            "wicket_kind": None,
            "wicket_player_out": None,
            "wicket_fielders": []
        }

        if "wickets" in delivery:
            for wicket in delivery["wickets"]:
                delivery_flat["wicket_kind"] = wicket.get("kind")
                delivery_flat["wicket_player_out"] = wicket.get("player_out")
                fielders = wicket.get("fielders", [])
                if isinstance(fielders, list):
                    delivery_flat["wicket_fielders"] = [
                        f.get("name") for f in fielders if isinstance(f, dict) and "name" in f
                    ]

        return delivery_flat

    def to_dataframe(self):
        """Convert all deliveries to a Pandas DataFrame."""
        return pd.DataFrame(self.t20_matches)


class MySQLInserter:
    def __init__(self, host, user, password, database):
        self.conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            charset='utf8mb4'
        )
        self.cursor = self.conn.cursor()

    def create_table(self):
        """Create t20_matches table in MySQL."""
        create_table_query = """
        CREATE TABLE IF NOT EXISTS t20_matches (
            id INT AUTO_INCREMENT PRIMARY KEY,
            match_file VARCHAR(255),
            team VARCHAR(100),
            over_no INT,
            batter VARCHAR(100),
            bowler VARCHAR(100),
            non_striker VARCHAR(100),
            runs_batter INT,
            runs_extras INT,
            runs_total INT,
            extras_type VARCHAR(50),
            wicket_kind VARCHAR(50),
            wicket_player_out VARCHAR(100),
            wicket_fielders TEXT
        );
        """
        self.cursor.execute(create_table_query)
        self.conn.commit()

    def insert_dataframe(self, df):
        """Insert DataFrame into MySQL table."""
        insert_query = """
        INSERT INTO t20_matches (
            match_file, team, over_no, batter, bowler, non_striker,
            runs_batter, runs_extras, runs_total, extras_type,
            wicket_kind, wicket_player_out, wicket_fielders
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        for _, row in df.iterrows():
            self.cursor.execute(insert_query, (
                row["match_file"],
                row["team"],
                row["over_no"],
                row["batter"],
                row["bowler"],
                row["non_striker"],
                row["runs_batter"],
                row["runs_extras"],
                row["runs_total"],
                row["extras_type"],
                row["wicket_kind"],
                row["wicket_player_out"],
                ",".join(row["wicket_fielders"]) if isinstance(row["wicket_fielders"], list) else None
            ))
        self.conn.commit()

    def close(self):
        """Close MySQL connection."""
        self.cursor.close()
        self.conn.close()


if __name__ == "__main__":
    extractor = MatchDataExtractor(folder_path="/Users/rangasamyelancheziyan/Desktop/cricket/t20s_json")
    extractor.extract_all_matches()
    df = extractor.to_dataframe()
    print(df.head()) 

    inserter = MySQLInserter(
        host="localhost",
        user="root",
        password="Sheela@1981",
        database="crickets_db"
    )
    inserter.create_table()
    inserter.insert_dataframe(df)
    inserter.close()

    print("✅ Data inserted into MySQL successfully!")

     match_file   team  over_no        batter            bowler   non_striker  \
0  1442989.json  India        0   YBK Jaiswal  C Wickramasinghe  Shubman Gill   
1  1442989.json  India        0   YBK Jaiswal  C Wickramasinghe  Shubman Gill   
2  1442989.json  India        0  Shubman Gill  C Wickramasinghe   YBK Jaiswal   
3  1442989.json  India        0  Shubman Gill  C Wickramasinghe   YBK Jaiswal   
4  1442989.json  India        0   YBK Jaiswal  C Wickramasinghe  Shubman Gill   

   runs_batter  runs_extras  runs_total extras_type wicket_kind  \
0            0            0           0        None        None   
1            1            0           1        None        None   
2            0            0           0        None        None   
3            1            0           1        None        None   
4            1            0           1        None        None   

  wicket_player_out wicket_fielders  
0              None              []  
1              None              [

In [13]:
import os
import json
import pandas as pd
import mysql.connector

class MatchDataExtractor:
    def __init__(self, folder_path):
        self.folder_path = folder_path
        self.ipl_matches = []

    def extract_all_matches(self):
        """Extract deliveries from all JSON files in the folder."""
        for filename in os.listdir(self.folder_path):
            if filename.endswith(".json"):
                file_path = os.path.join(self.folder_path, filename)
                self.extract_from_file(file_path, filename)

    def extract_from_file(self, file_path, filename):
        """Extract deliveries from a single match JSON file."""
        with open(file_path, 'r', encoding="utf-8") as f:
            data = json.load(f)

        for inning in data.get("innings", []):
            inning_data = inning if "team" in inning else list(inning.values())[0]
            team = inning_data.get("team")
            for over in inning_data.get("overs", []):
                over_number = over.get("over")
                for delivery in over.get("deliveries", []):  
                    self.ipl_matches.append(
                        self.flatten_delivery(delivery, team, over_number, filename)
                    )

    def flatten_delivery(self, delivery, team, over_number, filename):
        """Flatten delivery dictionary into a row."""
        extras_type = list(delivery.get("extras", {}).keys())
        extras_type_str = extras_type[0] if extras_type else None

        delivery_flat = {
            "match_file": filename,
            "team": team,
            "over_no": over_number,
            "batter": delivery.get("batter"),
            "bowler": delivery.get("bowler"),
            "non_striker": delivery.get("non_striker"),
            "runs_batter": delivery["runs"]["batter"],
            "runs_extras": delivery["runs"]["extras"],
            "runs_total": delivery["runs"]["total"],
            "extras_type": extras_type_str,
            "wicket_kind": None,
            "wicket_player_out": None,
            "wicket_fielders": []
        }

        if "wickets" in delivery:
            for wicket in delivery["wickets"]:
                delivery_flat["wicket_kind"] = wicket.get("kind")
                delivery_flat["wicket_player_out"] = wicket.get("player_out")
                fielders = wicket.get("fielders", [])
                if isinstance(fielders, list):
                    delivery_flat["wicket_fielders"] = [
                        f.get("name") for f in fielders if isinstance(f, dict) and "name" in f
                    ]

        return delivery_flat

    def to_dataframe(self):
        """Convert all deliveries to a Pandas DataFrame."""
        return pd.DataFrame(self.ipl_matches)


class MySQLInserter:
    def __init__(self, host, user, password, database):
        self.conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            charset='utf8mb4'
        )
        self.cursor = self.conn.cursor()

    def create_table(self):
        """Create ipl_matches table in MySQL."""
        create_table_query = """
        CREATE TABLE IF NOT EXISTS ipl_matches (
            id INT AUTO_INCREMENT PRIMARY KEY,
            match_file VARCHAR(255),
            team VARCHAR(100),
            over_no INT,
            batter VARCHAR(100),
            bowler VARCHAR(100),
            non_striker VARCHAR(100),
            runs_batter INT,
            runs_extras INT,
            runs_total INT,
            extras_type VARCHAR(50),
            wicket_kind VARCHAR(50),
            wicket_player_out VARCHAR(100),
            wicket_fielders TEXT
        );
        """
        self.cursor.execute(create_table_query)
        self.conn.commit()

    def insert_dataframe(self, df):
        """Insert DataFrame into MySQL table."""
        insert_query = """
        INSERT INTO ipl_matches (
            match_file, team, over_no, batter, bowler, non_striker,
            runs_batter, runs_extras, runs_total, extras_type,
            wicket_kind, wicket_player_out, wicket_fielders
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);
        """
        for _, row in df.iterrows():
            self.cursor.execute(insert_query, (
                row["match_file"],
                row["team"],
                row["over_no"],
                row["batter"],
                row["bowler"],
                row["non_striker"],
                row["runs_batter"],
                row["runs_extras"],
                row["runs_total"],
                row["extras_type"],
                row["wicket_kind"],
                row["wicket_player_out"],
                ",".join(row["wicket_fielders"]) if isinstance(row["wicket_fielders"], list) else None
            ))
        self.conn.commit()

    def close(self):
        """Close MySQL connection."""
        self.cursor.close()
        self.conn.close()


if __name__ == "__main__":
    extractor = MatchDataExtractor(folder_path="/Users/rangasamyelancheziyan/Desktop/cricket/ipl_json")
    extractor.extract_all_matches()
    df = extractor.to_dataframe()
    print(df.head())  

    inserter = MySQLInserter(
        host="localhost",
        user="root",
        password="Sheela@1981",
        database="crickets_db"
    )
    inserter.create_table()
    inserter.insert_dataframe(df)
    inserter.close()

    print("✅ Data inserted into MySQL successfully!")

     match_file                 team  over_no   batter    bowler  \
0  1426261.json  Sunrisers Hyderabad        0  TM Head  K Rabada   
1  1426261.json  Sunrisers Hyderabad        0  TM Head  K Rabada   
2  1426261.json  Sunrisers Hyderabad        0  TM Head  K Rabada   
3  1426261.json  Sunrisers Hyderabad        0  TM Head  K Rabada   
4  1426261.json  Sunrisers Hyderabad        0  TM Head  K Rabada   

       non_striker  runs_batter  runs_extras  runs_total extras_type  \
0  Abhishek Sharma            0            0           0        None   
1  Abhishek Sharma            0            0           0        None   
2  Abhishek Sharma            4            0           4        None   
3  Abhishek Sharma            0            0           0        None   
4  Abhishek Sharma            0            0           0        None   

  wicket_kind wicket_player_out wicket_fielders  
0        None              None              []  
1        None              None              []  
2       

In [14]:
import mysql.connector

class MySQLQueryRunner:
    def __init__(self, host, user, password, database):
        self.conn = mysql.connector.connect(
            host=host,
            user=user,
            password=password,
            database=database,
            charset='utf8mb4'
        )
        self.cursor = self.conn.cursor(dictionary=True)  # returns dict results

    def run_query(self, query):
        self.cursor.execute(query)
        return self.cursor.fetchall()

    def close(self):
        self.cursor.close()
        self.conn.close()

if __name__ == "__main__":
    runner = MySQLQueryRunner(
        host="localhost",
        user="root",
        password="Sheela@1981",
        database="crickets_db"
    )

    queries = {
        "Total runs scored by each team": """
            SELECT team, SUM(runs_total) AS total_runs
            FROM odi_matches
            GROUP BY team
            ORDER BY total_runs DESC;
        """,

        "Total runs scored by each batter": """
            SELECT batter, SUM(runs_batter) AS total_runs
            FROM odi_matches
            GROUP BY batter
            ORDER BY total_runs DESC
            LIMIT 10;
        """,

        "Total wickets fallen for each team": """
            SELECT team, COUNT(wicket_player_out) AS total_wickets
            FROM odi_matches
            WHERE wicket_player_out IS NOT NULL
            GROUP BY team
            ORDER BY total_wickets DESC;
        """,

        "Deliveries where extras were given": """
            SELECT * 
            FROM odi_matches
            WHERE runs_extras > 0;
        """,

        "Number of wickets by kind": """
            SELECT wicket_kind, COUNT(*) AS count
            FROM odi_matches
            WHERE wicket_kind IS NOT NULL
            GROUP BY wicket_kind
            ORDER BY count DESC;
        """,

        "Top bowlers by number of wickets taken": """
            SELECT bowler, COUNT(wicket_player_out) AS wickets
            FROM odi_matches
            WHERE wicket_player_out IS NOT NULL
            GROUP BY bowler
            ORDER BY wickets DESC
            LIMIT 10;
        """,

        "Runs scored in each over per team": """
            SELECT team, over_no, SUM(runs_total) AS runs_in_over
            FROM odi_matches
            GROUP BY team, over_no
            ORDER BY team, over_no;
        """,

        "Count of deliveries for each extras type": """
            SELECT extras_type, COUNT(*) AS deliveries_count
            FROM odi_matches
            WHERE extras_type IS NOT NULL
            GROUP BY extras_type;
        """,

        "Most common dismissal types": """
            SELECT wicket_kind, COUNT(*) AS frequency
            FROM odi_matches
            WHERE wicket_kind IS NOT NULL
            GROUP BY wicket_kind
            ORDER BY frequency DESC;
        """,

        "Total runs scored by each batter against each bowler": """
            SELECT batter, bowler, SUM(runs_batter) AS runs
            FROM odi_matches
            GROUP BY batter, bowler
            ORDER BY runs DESC
            LIMIT 20;
        """,

        "Number of deliveries bowled by each bowler": """
            SELECT bowler, COUNT(*) AS deliveries_bowled
            FROM odi_matches
            GROUP BY bowler
            ORDER BY deliveries_bowled DESC;
        """,

        "Batters caught out by specific fielder (e.g. Smith)": """
            SELECT wicket_player_out, wicket_fielders
            FROM odi_matches
            WHERE wicket_kind = 'caught'
            AND wicket_fielders LIKE '%Smith%';
        """,

        "Runs scored by batters excluding extras": """
            SELECT batter, SUM(runs_batter) AS runs_without_extras
            FROM odi_matches
            GROUP BY batter
            ORDER BY runs_without_extras DESC
            LIMIT 10;
        """,

        "Total extras conceded by each team": """
            SELECT team, SUM(runs_extras) AS extras_conceded
            FROM odi_matches
            GROUP BY team
            ORDER BY extras_conceded DESC;
        """,

        "Deliveries where wicket fell in last over (over_no=50)": """
            SELECT *
            FROM odi_matches
            WHERE wicket_player_out IS NOT NULL
            AND over_no = 50;
        """
    }

    for title, query in queries.items():
        print(f"\n--- {title} ---")
        results = runner.run_query(query)
        for row in results[:10]: 
            print(row)

    runner.close()



--- Total runs scored by each team ---
{'team': 'India', 'total_runs': Decimal('149013')}
{'team': 'Australia', 'total_runs': Decimal('137661')}
{'team': 'England', 'total_runs': Decimal('130219')}
{'team': 'Sri Lanka', 'total_runs': Decimal('120484')}
{'team': 'South Africa', 'total_runs': Decimal('116734')}
{'team': 'Pakistan', 'total_runs': Decimal('112022')}
{'team': 'New Zealand', 'total_runs': Decimal('108042')}
{'team': 'West Indies', 'total_runs': Decimal('98303')}
{'team': 'Bangladesh', 'total_runs': Decimal('75114')}
{'team': 'Zimbabwe', 'total_runs': Decimal('59109')}

--- Total runs scored by each batter ---
{'batter': 'V Kohli', 'total_runs': Decimal('14059')}
{'batter': 'KC Sangakkara', 'total_runs': Decimal('11618')}
{'batter': 'RG Sharma', 'total_runs': Decimal('10948')}
{'batter': 'MS Dhoni', 'total_runs': Decimal('10274')}
{'batter': 'AB de Villiers', 'total_runs': Decimal('9435')}
{'batter': 'TM Dilshan', 'total_runs': Decimal('9212')}
{'batter': 'LRPL Taylor', 'tot